Lecture 12 will cover how to tie the GUI-widgets to the application logic. It will cover the Model-View-Controller (MVC) design concept for GUI.

Reference
 * PyQt and Qt documentation

First a bit of standard stuff needed to work with Qt

In [1]:
#from PySide.QtCore import *
#from PySide.QtGui import *
#from PyQt4.QtCore import *
#from PyQt4.QtGui import *
from PyQt5.QtCore import *
from PyQt5.QtGui import *
from PyQt5.QtWidgets import *
import sys

In [2]:
qt_app = QApplication(sys.argv)

# Model / State

It is important to think of the *state* of an interactive piece of software.
In sequential scripts, it is easy to keep track of the state, as the order of events are all predetermined, but this is not the case in interactive applications (whether or not they are graphical).

For example, the minimum state of a chess game would be:
* Whose players turn it is.
* The position of all pieces left on the board.

but a nicely presented game might need to store more
* All the moves so far, in order, for each player
* The pieces taken by each player.

The user interface renders the information from the state. The user interacts with the user interface which sends signals back to the state to modify it.

## Mismatching UI and application state

A common mistake is to let the presented UI not match the actual program state.
Lets take the example of a simple painting application, where the state must at least include:
* The drawing surface (size, pixel values, etc.)
* The selected tool

When a tool is selected, it's commonly represented as a pressed down button.
When selecting a new tool, the previous tool button should be unpressed.
<img src="buttons.png"/>
Well, that's not to bad. We could even just unpress all the buttons before repressing the new one to be sure.
But, then, later on, we learn about the undo-operation and now there is a real risk of messing this up.


Another simple example would be the player name. What if, somewhere later on, we let the player change his name?
We would need to modify this in all occurrences where it might be printed.

The key difference here is that we either update
* the whole UI based on the *total* state
* a section of the UI based on the *difference* in the state
The first option can never display the wrong information, but the reason to sometimes prefer the second option is for performance.
Widget toolkits tend to only update the graphics whenever a drawing event occurs (window resized, mouse movement/clicks etc.)

It's therefore not uncommon to see applications where you need to "wiggle" the window edge around to force a window redraw event in order for the content to be updated. In games, this rarely happens, as games typically redraw, from the *total* state 60 frames per second, with little regard for battery life.

# Signals

Last lecture we looked (briefly) at procedures for placing widgets. I hope you all agree that it is fairly straight forward to construct these, and now the more difficult task of having all the buttons interact and actually do something.
One can read more about this on:
http://qt-project.org/wiki/Signals_and_Slots_in_PySide

In it's simplest form, we have a callback function

In [4]:
def some_func():
    print("some function has been called!")

def some_other_func():
    print("some other function has been called!")

qt_app = QApplication.instance()
button = QPushButton("Call some function")

button.clicked.connect(some_func)
#         ^               ^
#       signal           slot

button.clicked.connect(some_other_func) # We can connect multiple slots to a signal.

button.show()
qt_app.exec_()

some function has been called!
some other function has been called!
some function has been called!
some other function has been called!


0

Whenever the button is clicked, Qt finds out which button, and emits the signal for that button, like this:

In [5]:
button.clicked.emit(0)

some function has been called!
some other function has been called!


Signals can be emitted in other ways, for example using the `QTimer`.

# Model View Controller (MVC)

From wikipedia:

<img src="200px-MVC-Process.svg.png"/>

* A controller can send commands to the model to update the model's state (e.g., editing a document). It can also send commands to its associated view to change the view's presentation of the model (e.g., by scrolling through a document).
* A model notifies its associated views and controllers when there has been a change in its state. This notification allows the views to produce updated output, and the controllers to change the available set of commands. In some cases an MVC implementation might instead be "passive," so that other components must poll the model for updates rather than being notified.
* A view requests information from the model that it uses to generate an output representation to the user.

The model is presicely that of the *state* described earlier.

The controller is typically part of the *view*, e.g. selecting an object represented by some icon in the *view*.

## QStringListModel and QStringListView

A simple example is that of predefined QStringListModel and QStringListView. The QStringListView incorporates the role of the Controller as well (as expected).

First, lets try the normal list widgets to see the problem

In [6]:
qt_app = QApplication.instance()

data = ["one", "two", "three", "four", "five"]


combobox = QComboBox()
combobox.addItems(data)
combobox.show()


listWidget = QListWidget()
listWidget.addItems(data)
for i in range(listWidget.count()):
    item = listWidget.item(i)
    item.setFlags( item.flags() | Qt.ItemIsEditable )
listWidget.show()

qt_app.exec_()

0

As expected. When modifying the list widget, the information in the combobox does not update. This is because the widgets store copies of the data, and never modifies it.

In [7]:
qt_app = QApplication.instance()

data = ["one", "two", "three", "four", "five"]

model = QStringListModel(data)
   
combobox = QComboBox()
combobox.setModel(model)
combobox.show()
    
listView = QListView()
listView.setModel(model)
listView.show()

qt_app.exec_()

0

There are 2 components to this:
1. The views don't story copies of the data. The views always represent the state in the model.
2. The secondary view is updated as soon as the other is modified. 

The events go roughly like this:
 * The Views have a `setModel` (or equivalent) method which does 2 things
      - The View stores a reference to the model so that it can ask for the data it needs.
      - The View adds a slot to something like a "`dataChanged`"-signal in the Model to keep track of when data changes.
 * The user edits the first view window. The controller modifies the model by calling `setData` or equivalent.
 * The `setData` method in the Model emits a `dataChanged` signal (regardless of who modified the data).
 * The connected slots (callback methods) in every view is called, and each view will ask the model for whatever data they need.

One has to be careful as to not modify the data (state) without emitting the `dataChanged` signal (e.g. if you subclass QStringListModel)

# Subclassing and Model+View example

I prefer to subclass very frequently. There are many components in Qt that are meant to be subclassed, such as `QAbstractListModel`, which allows for convenient access to the signal system used in the `QStringListModel`, but for lists that might be of a different type (e.g. a list of chess pieces for a game).

Sometimes subclasses are just there to add a piece of useful information, or a small specialization. For example a simple QGraphicsScene that has a specialized background image:

In [8]:
class TableScene(QGraphicsScene):
    """ A scene with a table cloth background """
    def __init__(self):
        super().__init__()
        self.tile = QPixmap('cards/table.png')
        self.setBackgroundBrush(QBrush(self.tile))

And a QGraphicsSvgItem that also keeps track of it's local number (the number is really useful later when the user clicks on a the image)

In [9]:
from PyQt5.QtSvg import *

class CardSvgItem(QGraphicsSvgItem):
    """ A simple overloaded QGraphicsSvgItem that also stores the card position """
    def __init__(self, renderer, id):
        super().__init__()
        self.setSharedRenderer(renderer)
        self.position = id

And the big overloaded class, QGraphicsView. This is now a CardView widget, with a specialized constructor.
It prints the state from the given *player* model input. It has a callback function (no signals for now, keeping it simple) that it passes to `player` so that it can be informed when the state has changed.

*Note*, the view only gets a signal to indicate that *something has changed*. The view is responsible to ask the model for the information it needs to display to the user. The view might not need everything.

**NOTE**: This example doesn't rely on any card game library. It is not intended to be used in your assignment without several critical modifications.

In [10]:
class CardView(QGraphicsView):
    # Underscores indicate a private function!
    def __read_cards(): # Ignore the PyCharm warning on this line. It's correct.
        """
        Reads all the 52 cards from files.
        :return: Dictionary of SVG renderers
        """
        all_cards = dict()
        for suit in 'HDSC':
            for value in ['2', '3', '4', '5', '6', '7', '8', '9', '10', 'J', 'Q', 'K', 'A']:
                file = value + suit
                all_cards[file] = QSvgRenderer('cards/' + file + '.svg')
        return all_cards

    # We read all the card graphics as static class variables:
    back_card = QSvgRenderer('cards/Red_Back_2.svg')
    all_cards = __read_cards()

    def __init__(self, player, card_spacing=250, padding=10):

        self.scene = TableScene()
        super().__init__(self.scene)

        self.card_spacing = card_spacing
        self.padding = padding

        self.player = player
        player.set_callback(self.change_cards)

        self.change_cards() # Add the cards the first time around to represent the initial state.

    def change_cards(self):
        # Add the cards from scratch:
        self.scene.clear()
        for i, c_ref in enumerate(self.player.cards):
            renderer = self.back_card if self.player.marked_cards[i] else self.all_cards[c_ref]

            c = CardSvgItem(renderer, i)
            self.scene.addItem(c)

        self.update_view()

    def update_view(self):
        for c in self.scene.items():
            # Lets have the cards take up almost the (current) full height
            card_height = c.boundingRect().bottom()
            scale = (self.height()-2*self.padding)/card_height

            c.setPos(c.position * self.card_spacing*scale, 0)
            c.setScale(scale)
            #c.setOpacity(0.5 if self.player.marked_cards[c.position] else 1.0)

        # Put the scene bounding box
        self.scene.setSceneRect(-self.padding, -self.padding, self.viewport().width(), self.viewport().height())


    def resizeEvent(self, painter):
        # If the widget is resize, we gotta adjust the card sizes.
        # QGraphicsView automatically re-paints everything when we modify the scene.
        self.update_view()
        super().resizeEvent(painter)

    # This is the Controller part of the GUI, handling input events that modify the Model
    def mousePressEvent(self, event):
        #item = self.scene.itemAt(event.pos()) # For PyQt4
        item = self.scene.itemAt(event.pos(), self.transform())
        if item is not None:
            # Report back that the Model that the user marked a given position:
            self.player.mark_position(item.position)

Lets put in a simple *Model* for now. Something which represent a player state:

In [11]:
class Player:
    def __init__(self):
        # Lets use some hardcoded values for most of this to start with:
        self.cards = ['QS', 'AD', '7C']
        self.marked_cards = [False]*len(self.cards)
        self.credits = 100
        self.folded = False
        self.cb = None

    def set_callback(self, cb):
        # Instead of the sophisticated signal system, I have a simple callback here.
        # This only works if there is just one viewer!
        # But I want to reduce the complexity in this example to make it clear why things occur.
        self.cb = cb

    def active(self):
        return credits > 0 and not self.folded

    def mark_position(self, i):
        # Mark the card as position "i" to be thrown away
        self.marked_cards[i] = not self.marked_cards[i]
        if self.cb is not None: self.cb()

    def marked(self, id):
        return self.marked_cards[id]

In [12]:
# Lets test it out
qt_app = QApplication.instance()
p = Player()
card_view = CardView(p)
box = QVBoxLayout()
box.addWidget(card_view)
player_view = QGroupBox("Player 1")
player_view.setLayout(box)

player_view.show()

qt_app.exec_()

0

# Summary

* You should not need to refer to the *view* to determine the state of the game. It should be isolated neatly in a separate *model* (e.g. the state of the game). I have seen people try to read the labels of a GUI widget to determine the state of the game. 
* The *model* should (probably) tell the *view* when something has changed (and nothing more).
* Don't try to optimize to only do partial updates. Keep it simple and just tell all the views to update everything instead of trying to do partial updates. Computers are fast and probing the model for a few values is likely very fast compared to drawing the UI.

# Simple game example (live demo)

This purpose of this game is to fight over the computer, and manage to click your button 10 times before the opponent. For this we have 2 buttons and 2 labels. We utilize Qt signals to communicate between the model and the view/controller.

In [13]:
class AddOneGame(QObject):
    
    new_total = pyqtSignal()
    winner = pyqtSignal(str,)
    
    def __init__(self):
        super().__init__()
        self.players = ['Micke','Thomas']
        self.total = [0,0]
    
    def player_click(self, ind):
        self.total[ind] += 1
        self.new_total.emit()
        self.check_winner()
    
    def reset(self):
        self.total = [0, 0]
        self.new_total.emit()
    
    def check_winner(self):
        if self.total[0] >= 10:
            self.winner.emit(self.players[0] + " won!")
            self.reset()
        elif self.total[1] >= 10:
            self.winner.emit(self.players[1] + " won!")
            self.reset()

In [14]:
class GameView(QWidget):
    def __init__(self, game_model):
        super().__init__()
        buttons = [QPushButton(game_model.players[0]), QPushButton(game_model.players[1])]
        self.labels = [QLabel(), QLabel()]
        vbox = QVBoxLayout()
        vbox.addWidget(buttons[0])
        vbox.addWidget(self.labels[0])
        vbox.addWidget(buttons[1])
        vbox.addWidget(self.labels[1])
        
        reset_button = QPushButton('Reset')
        reset_button.clicked.connect(game_model.reset)
        
        vbox.addWidget(reset_button)
        
        self.setLayout(vbox)
        self.show()
        
        # Model!
        self.game = game_model
        self.update_labels()
        game_model.new_total.connect(self.update_labels)
        game_model.winner.connect(self.alert_winner)
        
        # Controller!
        def player0_click(): game_model.player_click(0)
        buttons[0].clicked.connect(player0_click)
        def player1_click(): game_model.player_click(1)
        buttons[1].clicked.connect(player1_click)
        
    def update_labels(self):
        for i in range(2): self.labels[i].setText(str(self.game.total[i]))

    def alert_winner(self, text):
        msg = QMessageBox()
        msg.setText(text)
        msg.exec()

In [15]:
qt_app = QApplication.instance()
game = AddOneGame()
view = GameView(game)
qt_app.exec_()

0